In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import json

sys.path.append(r"C:\Users\janbe\Documents\GitHub Repos\pulse")
from src.eval.llm_analyzer import LLMAnalyzer
from src.eval.metrics import calculate_all_metrics

In [8]:
mdata_path = [
    r"C:\Users\janbe\Documents\GitHub Repos\pulse\output_to_keep\results_benchmark\llms\gemini2p5flash\20250624_101940_gemini2p5flash_zhu_b_zero_shot\Gemini2p5flash_sepsis_miiv_20250624_101940_metadata.csv"
]

output_path = os.path.dirname(mdata_path[0])

df_mdata = LLMAnalyzer.load_metadata(mdata_path)

Model Name: Gemini2p5flash, Task: sepsis, Dataset: miiv, Timestamp: 20250624_101940


In [9]:
df_mdata

,Input Prompt,Target Label,Predicted Probability,Predicted Diagnosis,Predicted Explanation,Tokenization Time,Inference Time,Input Tokens,Output Tokens,Thinking Tokens,Thinking Output,model_name,task,dataset,timestamp
0,You are an experienced doctor in Intensive Car...,1,0.20,not-diagnosis,While the patient exhibits an elevated CRP (44...,0.0,1.665101,3980,136,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
1,You are an experienced doctor in Intensive Car...,1,0.15,not-diagnosis,The patient's vital signs and lab values at th...,0.0,2.021487,3979,185,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
2,You are an experienced doctor in Intensive Car...,1,0.15,not-diagnosis,The patient exhibits some mild abnormalities s...,0.0,2.870357,3977,215,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
3,You are an experienced doctor in Intensive Car...,1,0.10,not-diagnosis,The patient's vital signs and lab values are l...,0.0,2.321771,3978,191,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
4,You are an experienced doctor in Intensive Car...,1,0.10,not-diagnosis,The patient's heart rate is slightly elevated ...,0.0,2.105629,3978,142,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,You are an experienced doctor in Intensive Car...,1,0.20,not-diagnosis,The patient's SOFA score does not show an incr...,0.0,1.753384,3963,146,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
986,You are an experienced doctor in Intensive Car...,1,0.15,not-diagnosis,The patient's vital signs and lab values are l...,0.0,2.108118,3963,146,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
987,You are an experienced doctor in Intensive Car...,1,0.10,not-diagnosis,The patient's vital signs and lab values remai...,0.0,1.829313,3963,139,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940
988,You are an experienced doctor in Intensive Car...,1,0.20,not-diagnosis,The patient's vital signs and lab values at th...,0.0,1.818219,3963,173,0,NaN,Gemini2p5flash,sepsis,miiv,20250624_101940


In [10]:
results = []
prompting_id = "zhu_2024b_zero_shot_preprocessor"


metrics = calculate_all_metrics(
    y_true=df_mdata["Target Label"],
    y_pred=df_mdata["Predicted Probability"],
)
model_id = df_mdata["model_name"].iloc[0]
task_id = df_mdata["task"].iloc[0]
dataset = df_mdata["dataset"].iloc[0]
results.append(metrics)

run_id = df_mdata["timestamp"].iloc[0]

metrics_overall = {
    k: float(v)
    for k, v in metrics.items()
    if k
    in [
        "auroc",
        "auprc",
        "normalized_auprc",
        "specificity",
        "f1_score",
        "accuracy",
        "balanced_accuracy",
        "precision",
        "recall",
        "mcc",
        "kappa",
        "minpse",
    ]
}

output_dict = {
    "model_id": model_id,
    "task_id": task_id,
    "dataset": dataset,
    "prompting_id": prompting_id,
    "run_id": run_id,
    "metrics_summary": {"overall": metrics_overall},
}

metrics_report_path = os.path.join(
    output_path, f"{model_id}_metrics_report.json"
)
if os.path.exists(metrics_report_path):
    with open(metrics_report_path, "r") as f:
        try:
            existing_data = json.load(f)
            if isinstance(existing_data, list):
                existing_data.append(output_dict)
            else:
                existing_data = [existing_data, output_dict]
        except json.JSONDecodeError:
            existing_data = [output_dict]
    with open(metrics_report_path, "w") as f:
        json.dump(existing_data, f, indent=2)
else:
    with open(metrics_report_path, "w") as f:
        json.dump(output_dict, f, indent=2)

df_metrics = pd.DataFrame(results)
df_metrics

,auroc,auprc,normalized_auprc,specificity,f1_score,accuracy,balanced_accuracy,precision,recall,mcc,kappa,minpse
0,0.442,0.02,0.788,0.741,0.022,0.726,0.431,0.012,0.12,-0.05,-0.026,0.026
